In [2]:
df = read.csv("~/Downloads/Investors day data - eventbrite export 041118 .csv", stringsAsFactors = FALSE)
investors = read.csv("~/Downloads/Clean Investors Day Data - Investors(2).csv", stringsAsFactors = FALSE)
startups = df[df$Ticket.Type == "Startup", c(9, 11, 12, 13, 14, 15, 16, 17, 18)]
startups = startups[grep("Getting investment", startups$What.are.you.interested.in.), ]

In [3]:
startuplist = lapply(gsub(" ", "", startups$What.is.your.field.),function(y){strsplit(as.character(y), fixed = TRUE, "|")})
investorlist = lapply(gsub(" ", "", investors$Field.of.interest),function(y){strsplit(as.character(y), fixed = TRUE, "|")})

startupdf = data.frame(matrix(0, ncol = length(unique(unlist(startuplist))), nrow = nrow(startups)))
investordf = data.frame(matrix(0, ncol = length(unique(unlist(startuplist))), nrow = nrow(investors)))
colnames(startupdf) = unique(unlist(startuplist)); colnames(investordf) = unique(unlist(startuplist))

for (i in 1:length(startuplist)){for (j in 1:length(startuplist[[i]][[1]])){startupdf[i, startuplist [[i]][[1]][j]] = 1}}
for (i in 1:length(investorlist)){for (j in 1:length(investorlist[[i]][[1]])){investordf[i, investorlist[[i]][[1]][[j]]] = 1}}

In [12]:
listofrankings = list()

for (i in 1:nrow(investordf)){
  relevant = startupdf[, as.numeric(investordf[i,1:48 ]) == 1]
  x = matrix(unlist(sort(rowSums(scale(center = FALSE, relevant)), 
                         decreasing = TRUE, index.return = 1)), nrow = nrow(startups))
  x = data.frame(x[,1], startups[x[,2], c(1,3,5,8)])
  names(x)[1:2] = c("Score", "Matched Startup")
  listofrankings[[i]] = x
}

matches = gs_new(title = "Investor Matches")
for (i in 1:length(listofrankings)){
  matches = matches %>% 
    gs_ws_new(ws_title = ifelse(nchar(investors[i,1]) <= 30, investors[i,1], substr(investors[i,1], 1, 30)), 
              input = listofrankings[[i]],trim = TRUE, verbose = FALSE)}